# First Project for the Social Computing Course

We are gonna start to import all the stuff we need

In [1]:
import tweepy
import os
import json
import pprint
import csv

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pylab import rcParams

Now to use the API of Twitter, we need an developer account.  
So please create one and add the keys to continue use this notebook.

In [ ]:
# Twitter Credentials
api_key = input("Enter your api key: ")
api_key_secret = input("Enter your api key secret: ")
access_token = input("Enter your access token: ")
access_token_secret = input("Enter your access token secret: ")
bearer_token = input("Enter your bearer token: ")

Once you have done, it is time to create the wrapper of Tweepy to make requestes to the API

In [4]:
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
if (api.verify_credentials):
  print("Authentication success!")
else:
  print("Got a error")

Authentication success!
